In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import  VotingClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [3]:
df=pd.read_csv("./dataset/train_clean_data.csv",index_col=0)
df.head()

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,gender_Male,married_Yes,education_Not Graduate,property_area_Semiurban,property_area_Urban,self_employed_Yes,Loan_status_Y
1,LP001003,1,4583,1508.0,128.0,360.0,1.0,1,1,0,0,0,0,0
2,LP001005,0,3000,0.0,66.0,360.0,1.0,1,1,0,0,1,1,1
3,LP001006,0,2583,2358.0,120.0,360.0,1.0,1,1,1,0,1,0,1
4,LP001008,0,6000,0.0,141.0,360.0,1.0,1,0,0,0,1,0,1
5,LP001011,2,5417,4196.0,267.0,360.0,1.0,1,1,0,0,1,1,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 517 entries, 1 to 613
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Loan_ID                  517 non-null    object 
 1   Dependents               517 non-null    object 
 2   ApplicantIncome          517 non-null    int64  
 3   CoapplicantIncome        517 non-null    float64
 4   LoanAmount               517 non-null    float64
 5   Loan_Amount_Term         517 non-null    float64
 6   Credit_History           517 non-null    float64
 7   gender_Male              517 non-null    int64  
 8   married_Yes              517 non-null    int64  
 9   education_Not Graduate   517 non-null    int64  
 10  property_area_Semiurban  517 non-null    int64  
 11  property_area_Urban      517 non-null    int64  
 12  self_employed_Yes        517 non-null    int64  
 13  Loan_status_Y            517 non-null    int64  
dtypes: float64(4), int64(8), object

In [7]:
df.shape

(517, 14)

In [9]:
df['Loan_status_Y'].value_counts()

Loan_status_Y
1    360
0    157
Name: count, dtype: int64

In [11]:
''' Train Test Split'''
X=df.drop(['Loan_ID','Dependents','Loan_status_Y'],axis=1)
y=df['Loan_status_Y']

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test=train_test_split(X,y, test_size=0.33,stratify=y,random_state=42)

In [15]:
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.7 MB 10.9 MB/s eta 0:00:01
   --- ------------------------------------ 0.8/10.7 MB 10.2 MB/s eta 0:00:01
   ----- ---------------------------------- 1.3/10.7 MB 10.6 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/10.7 MB 10.5 MB/s eta 0:00:01
   -------- ------------------------------- 2.2/10.7 MB 10.8 MB/s eta 0:00:01
   ---------- ----------------------------- 2.8/10.7 MB 10.4 MB/s eta 0:00:01
   ------------ --------------------------- 3.3/10.7 MB 10.5 MB/s eta 0:00:01
   -------------- ------------------------- 3.8/10.7 MB 10.4 MB/s eta 0:00:01
   --------------- ------------------------ 4.2/10.7 MB 10.3 MB/s eta 0:00:01
   ----------------- ---------------------- 4.7/10.7 MB 10.4 MB/s eta 0:00:01
   ------------------- -------------------- 5.2/10.7 MB 10.3 MB/s eta 0:00:01
   --------------------- ------------------ 5.7/10.7 MB 10.4 MB/s eta 0:

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.


  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl (11.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.0
    Uninstalling scikit-learn-1.7.0:
      Successfully uninstalled scikit-learn-1.7.0


**Synthetic Minority Over-sampling Technique**
- using SMOTE (Synthetic Minority Over-sampling Technique) from the imblearn library to balance an imbalanced dataset
- The target **y_train** has **at least one minority class with fewer samples** than the majority class — otherwise SMOTE has nothing to balance.
- If you have **categorical features in X_train,** consider using **SMOTENC** instead of SMOTE, as regular SMOTE is designed for numerical data only.
- SMOTE for Nominal and Continuous features. It allows you to handle mixed-type data (numerical + categorical) during oversampling.
- SMOTE creates new, synthetic samples by **interpolating**(finding a point between two known points.) between existing ones.

In [18]:
from imblearn.over_sampling import SMOTE
'''set the random state, it ensures that the synthetic data generated during the oversampling process is the same every time you run the code'''
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [20]:
X_train_res.shape

(482, 11)

In [22]:
y_train_res.shape

(482,)

**Voting classifier**

In [25]:
clf1=DecisionTreeClassifier(class_weight='balanced')
clf2=GaussianNB()
clf3=LogisticRegression(solver='liblinear',max_iter=200,class_weight='balanced')

In [27]:
eclf1=VotingClassifier(estimators=[('DT',clf1),('GNB',clf2),('LR',clf3)], voting='hard')
eclf1=eclf1.fit(X_train_res,y_train_res)

In [29]:
predictions=eclf1.predict(X_test)

In [31]:
predictions

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0], dtype=int64)

In [33]:
from sklearn.metrics import classification_report , confusion_matrix

print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.73      0.62      0.67        52
           1       0.84      0.90      0.87       119

    accuracy                           0.81       171
   macro avg       0.78      0.76      0.77       171
weighted avg       0.81      0.81      0.81       171

[[ 32  20]
 [ 12 107]]


In [35]:
eclf2=VotingClassifier(estimators=[('DT',clf1),('GNB',clf2),('LR',clf3)], voting='soft')
eclf2=eclf2.fit(X_train,y_train)

In [37]:
prediction=eclf2.predict(X_test)

In [39]:
from sklearn.metrics import classification_report , confusion_matrix

print(classification_report(y_test,prediction))
print(confusion_matrix(y_test,prediction))

              precision    recall  f1-score   support

           0       0.67      0.60      0.63        52
           1       0.83      0.87      0.85       119

    accuracy                           0.79       171
   macro avg       0.75      0.74      0.74       171
weighted avg       0.78      0.79      0.79       171

[[ 31  21]
 [ 15 104]]
